In [1]:
from exp.utils import *
from exp.models import *
from exp.losses import *
from tqdm.notebook import tqdm
from multiprocessing import Pool

import torch
import torch.nn as NN
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [2]:
picked_labels = ["Atelectasis"]#["Atelectasis", "Cardiomegaly", "Pneumonia"]
already_pretrained_labels = ["Atelectasis"]#["Cardiomegaly", "Pneumonia"]
already_pretrained_lrs = [1e-11]#[1e-10, 1e-10]
already_pretrained = {k:v for k,v in zip(already_pretrained_labels, already_pretrained_lrs)}
already_trained = []#["Atelectasis"]
train_labels = [l for l in picked_labels if l not in already_trained]

In [3]:
architecure = "MNASNET1_v1"

In [4]:
for label in sorted(train_labels):
    print(f"Training model to classify '{label}'")
    
    # Seed
    seed = 92
    seed_everything(seed)
    
    # Inital setup
    model_name = f"{architecure}_{label}"#f"sam_densenet_v1_{label}"
    #model_type = "densenet"
    bs = 16
    lr = 1e-3
    epochs = 50
    patience = 1
    rho = 0.05
    with_reset = False
    image_size = (224, 224)
    device = get_device()
    labels = get_labels()
    
    # Load data
    train_df, valid_df, test_df = get_dataframes(include_labels=labels, 
                                                 small=False)
    print(train_df.shape, valid_df.shape, test_df.shape)
    train_df = get_binary_df(label, train_df)
    valid_df = get_binary_df(label, valid_df)
    test_df = get_binary_df(label, test_df)
    
    # Compute label weights
    train_label = train_df[[label]].values
    neg_weights, pos_weights = compute_class_freqs(train_label)
    neg_weights, pos_weights = torch.Tensor(neg_weights), torch.Tensor(pos_weights)
    print(neg_weights, pos_weights)
    
    # Get transforms
    train_tfs, test_tfs = get_transforms(image_size=image_size)
    
    # Create datasets
    train_ds = CRX8_Data(train_df, get_image_path(), label, image_size=image_size, transforms=train_tfs)
    valid_ds = CRX8_Data(valid_df, get_image_path(), label, image_size=image_size, transforms=test_tfs)
    test_ds  = CRX8_Data(test_df , get_image_path(), label, image_size=image_size, transforms=test_tfs)
    
    # Create dataloaders
    train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs, shuffle=False)
    test_dl  = DataLoader(test_ds,  batch_size=bs, shuffle=False)
    dataloaders = {
        "train": train_dl,
        "val": valid_dl,
        "test": test_dl
    }
    
    # Load imagenet-pretrained model
    if label in list(already_pretrained.keys()):
        model = load_model(model_name)
        lr = already_pretrained[label]
        print("Loaded pretrained model!")
    else:
        model = pretrained_mnasnet1_0()
    model = model.to(device)
    
    
    
    # Get criterion and optimizer
    criterion = get_weighted_loss_with_logits(pos_weights.to(device), 
                                              neg_weights.to(device))
    sam_optimizer = SAM(model.parameters(), torch.optim.Adam, lr=lr, rho=rho)
    
    # Train model
    model, history = fit(model, criterion, sam_optimizer, 
                         dataloaders, model_name, epochs, 
                         lr, sam=True, with_reset=with_reset, metric="loss", patience=patience)

FERTIG()

Training model to classify 'Atelectasis'
Using the GPU!
(69219, 25) (17305, 25) (25596, 24)
tensor([0.0961]) tensor([0.9039])
Epoch 1:



Train: Loss: 0.102, Acc: 0.647, AUROC: 0.755



Val: Loss: 0.210, Acc: 0.692, AUROC: 0.769
Saved model with loss 0.2096
Epoch 2:



Train: Loss: 0.097, Acc: 0.681, AUROC: 0.787



Val: Loss: 0.279, Acc: 0.709, AUROC: 0.768
Lowered lr to 0.0001
Epoch 3:


...


Train: Loss: 0.081, Acc: 0.741, AUROC: 0.859



Val: Loss: 0.105, Acc: 0.729, AUROC: 0.806
Lowered lr to 1.0000000000000003e-10
Epoch 14:



Train: Loss: 0.080, Acc: 0.745, AUROC: 0.865



Val: Loss: 0.102, Acc: 0.685, AUROC: 0.802
Lowered lr to 1.0000000000000003e-11
Epoch 15:


KeyboardInterrupt: 